# **Tree-Based Methods**

## Linear Decision Boundary with Logistic Regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs, make_circles, make_classification, fetch_olivetti_faces
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
import pandas as pd

%matplotlib inline

In [ ]:
def plot_decision_boundary(X, y, clf):
    x0_mesh, x1_mesh = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, 0.1),
                                 np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, 0.1))

    y_mesh_pred = clf.predict(np.c_[x0_mesh.ravel(), x1_mesh.ravel()])
    y_mesh_pred = y_mesh_pred.reshape(x0_mesh.shape)

    plt.contourf(x0_mesh, x1_mesh, y_mesh_pred, cmap="RdBu", vmin=0, vmax=1)
    plt.scatter(X[:, 0], X[:, 1], c = y, s = 50, edgecolor = 'w', cmap="RdBu", vmin=-.2, vmax=1.2, linewidth=1)
    plt.show()

In [ ]:
X, y = make_classification(n_samples=1000, 
                           n_features=2, 
                           n_redundant=0, 
                           n_informative=2, 
                           random_state=10, 
                           n_clusters_per_class=1)

x_train = X[:500,:]
x_test = X[500:,:]
y_train = y[:500]
y_test = y[500:]

In [ ]:
plt.scatter(x_train[:, 0], x_train[:, 1], c = y_train, s = 50, edgecolor = 'w', cmap="RdBu", vmin=-.2, vmax=1.2, linewidth=1)

In [ ]:
clf = LogisticRegression().fit(x_train, y_train)
train_acc = clf.score(x_train,y_train)
test_acc = clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')
plot_decision_boundary(x_test,y_test,clf)

## Non-Linear Decision Boundary with Logistic Regression

In [ ]:
X, y = make_circles(n_samples=1000, noise=0.2, factor=0.01, random_state=1000)

x_train = X[:500,:]
x_val = X[500:600,:]
x_test = X[600:,:]
y_train = y[:500]
y_val = y[500:600]
y_test = y[600:]

In [ ]:
plt.scatter(x_train[:, 0], x_train[:, 1], c = y_train, s = 50, edgecolor = 'w', cmap="RdBu", vmin=-.2, vmax=1.2, linewidth=1)

In [ ]:
clf = LogisticRegression().fit(x_train, y_train)
train_acc = clf.score(x_train,y_train)
test_acc = clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')
plot_decision_boundary(x_test,y_test,clf)

## Decision Trees

* Decision Trees can be applied to both classification and regression problems.
* Can be thought of as partitioning the data space into into regions of similar points. Within those regions we aggregate results via voting (classification) or averaging (regression).

### CART Algorithm
To choose the first split:
1. Sort all of the observed values from your training data for each predictor variable. Between every two values is a possible split point.
2. Evaluate the performance of choosing each possible split point. 
    - For regression, this would imply averaging each training observation that would fall into either partition. This averaged value would be the predicted value of each observation in that partition and from this the MSE could be calculated.
    - For classification, the process is identical except Gini impurity is used as the evaluation metric.
3. The split point that results in the largest improvement of the evaluation metric is chosen.
4. Repeat this process until some stopping criterion is reached.
5. The terminal nodes (or leaves) of the tree will be the average or voted label of the training observations that fall into it. These determine the predicted value of a given observation.

### Common Stopping Criterion
- Max depth: The maximum depth of a tree where the "depth" is some number of consecutive partitions.
- Minimum samples to split: The minimum number of samples necessary in a node for that node to be split on.
- Minimum samples in a leaf node: The minimum number of samples that must be in each terminal (leaf) node for a split to be performed.
- Minimum impurity decrease: A node will be split if it decreases the impurity beyond a predefined threshold. This helps avoid unnecessary splits.

In [ ]:
clf = DecisionTreeClassifier().fit(x_train, y_train)
train_acc = clf.score(x_train,y_train)
test_acc = clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')
plot_decision_boundary(x_test,y_test,clf)

### Pruning Trees

If allowed to grow completely (i.e. no stopping criterion), a Decision Tree can achieve perfect performance (aside from any inherent measurement error). This often comes at the cost of the ability to generalize to unseen data, an important feature of predictive modeling. A smaller tree might lead to lower variance (less overfitting) at the cost of a slight increase in bias (increased underfitting). 

Choosing a stopping criterion would avoid this issue but it can be a bit too shortsighted. Using minimum impurity decrease, an early split might seem worthless but may lead to a very good split later on that might be missed if it doesn't exceed the impurity decrease threshold. We'll see an example of this with the XOR problem later on.

Rather than choosing a stopping criterion a priori though, a more common approach is to "prune" a tree after it is grown to some large limit. Pruning involves recursively removing nodes to end up with a smaller subtree that should ideally generalize better to unseen data.

#### Cost Complexity Pruning

The most common method of pruning is referred to as cost complexity pruning (CCP), it also known as weakest link pruning. The complexity parameter, $\alpha$ is used to define the cost-complexity measure, $R_{\alpha}(T)$, of a given tree $T$, where

$$
R_{\alpha}(T) = R(T) + \alpha \lvert T \rvert
$$

where $\lvert T\rvert$ indicates the number of terminal nodes and $R(T)$ is the error rate of the tree. The cost complexity measure of a single node is $R_{\alpha}(t) = R(t) + \alpha$. The branch, $T_t$, is defined to be a tree where node $t$ is its root. In general, the impurity of a node is greater than the sum of impurities of its terminal nodes, $R(T_t) < R(t)$. However, the cost complexity measure of a node, $t$, and its branch, $T_t$, can be equal depending on $\alpha$. We define the effective $\alpha$ of a node to be the value where they are equal, $R_{\alpha}(T_t)=R_{\alpha}(t)$ or

$$
\alpha_{eff}(t)=\frac{R(t)-R(T_t)}{\lvert T \rvert -1}
$$

A non-terminal node with the smallest effective $\alpha$ is the weakest link and will be pruned.

Source: https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning

In [ ]:
path = clf.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

plt.plot(ccp_alphas, impurities)
plt.xlabel("effective alpha")
plt.ylabel("total impurity of leaves")

In [ ]:
impurities

This plot shows us the relationship between $\alpha$ and the impurity of the leaves on the training data. Similarly, we can look at how those $\alpha$'s relate to the depth of the trees.

In [ ]:
clfs = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(x_train, y_train)
    clfs.append(clf)
    
tree_depths = [clf.tree_.max_depth for clf in clfs]
plt.figure(figsize=(10,  6))
plt.plot(ccp_alphas[:-1], tree_depths[:-1])
plt.xlabel("effective alpha")
plt.ylabel("total depth")

These effective $\alpha$ values were determined based on the training set, meaning that they will be susceptible to overfitting stiil. To counter this, we compare different subtrees for the various values of $\alpha$ with the resulting accuracy on a holdout validation set.

Note: This is often done via cross-validation but is simply shown here with a single validation set for brevity.

In [ ]:
acc_scores = [clf.score(x_val, y_val) for clf in clfs]

tree_depths = [clf.tree_.max_depth for clf in clfs]
plt.grid()
plt.plot(ccp_alphas[:-1], acc_scores[:-1])
plt.xlabel("alpha")
plt.ylabel("Accuracy scores")

In [ ]:
optimal_clf = clfs[np.argmax(acc_scores)]

train_acc = optimal_clf.score(x_train,y_train)
test_acc = optimal_clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

plot_decision_boundary(x_test,y_test,optimal_clf)

### Trees are not always optimal (XOR)

Decision Trees are what is referred to as a greedy algorithm, since they will take a nearsighted view of splitting, selecting the best available split without considering the future. This can lead to non-optimal results as evidenced in the simple example of the XOR problem, where there the optimal splits would be at 0 on either dimension but either of those splits from the root node would lead to an error rate of about 50%.

In [ ]:
def xor(n=1000):
    X = (-1 - 1) * np.random.random_sample((n,2)) + 1
    
    y = []
    for i in range(X.shape[0]):
        if X[i,0] <= 0 and X[i,1] > 0 or X[i,0] > 0 and X[i,1] <= 0:
            y.append(1)
        else:
            y.append(0)
    return X, y

X, y = xor()
plt.scatter(X[:, 0], X[:, 1], c = y, s = 50, edgecolor = 'w', cmap="RdBu", vmin=-.2, vmax=1.2, linewidth=1)

In [ ]:
clf = DecisionTreeClassifier().fit(X, y)
train_acc = clf.score(X,y)

print(f'Training Accuracy: {train_acc}')
plot_decision_boundary(X,y,clf)

In [ ]:
plt.figure(figsize=(10, 10))
plot_tree(clf)
plt.show()

### Pros and Cons of Decision Trees
Pros:
- Easy to interpret
- Able to handle both numerical and categorical
- Doesn't require much data prep (e.g. no need to normalize values)
- Can be handed off to non-data people and still be understood

Cons:
- Low bias but high variance.
    - Very prone to overfitting
    - A slight change in the inputs can result in significantly different tree structure
- For categorical variables, information gain in Decision Trees is highly biased towards variables with a higher number of levels.
- Time complexity isn't great. During training trees are $O(nlogn * d)$

## Bias-Variance Tradeoff

In [ ]:
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=100, n_clusters_per_class=1
)

x_train = X[:50,:]
x_test = X[50:,:]
y_train = y[:50]
y_test = y[50:]

clf = DecisionTreeClassifier().fit(X, y)
train_acc = clf.score(x_train,y_train)
test_acc = clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')
plot_decision_boundary(x_test,y_test,clf)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Bias_and_variance_contributing_to_total_error.svg/2560px-Bias_and_variance_contributing_to_total_error.svg.png" width=400 height=400 />

<img src="https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/images/bias_variance/bullseye.png" height=400 width=400 />

# Random Forest

## Bootstrap Aggregating (Bagging)
As stated before, Decision Trees are very prone to overfitting, such that a small change in the inputs can have a dramatic effect on the tree that is grown. One way to counter this would be to train the Decision Tree on a bootstrapped sample of the data. A bootstrap sample is a dataset that is sampled with replacement to be as large as the original data. We can create infinitely many bootstrap samples and thus train infinitely many different trees on those samples. Each of these trees, however, would still exhibit high variance (overfit) to their respective bootstrap sample.

By aggregating the predictions from each tree together though, we should expect to see a decrease in the variance. This is the origin of the term Bagging a portmanteau of Bootstrap and Aggregating. Bagging can be utilized with almost any Machine Learning method, even Linear Regression, but it has been shown to be especially effective with Decision Trees.

## Random Forests
Random Forests (Breiman, 2001) take this idea of Bagging one step further though, with a goal of further reducing the variance with little to no increase in bias. The issue with simply fitting trees using bagging is that each tree is still being trained on the same set of predictors, so, assuming a true random sample, these trees won't necessarily be significantly different than each other. Errors in one tree will likely be present in many others, so Random Forests seek to decorrelate the trees even further.

The algorithm is as follows:
1. Fit a large number of Decision Trees on bootstrapped samples of the training data.
2. For each split, rather than considering every possible variable to split on, randomly select a subset of predictor variables and only sort and evaluate the splits along those variables.
3. Grow each tree to max depth (to reduce bias)
4. Vote (classification) or average (regression) each predicted value across all trees to perform the final prediciton.

These decorrellated trees typically show significantly reduced variance without increasing the low bias that is common in Decision Trees.

Random Forests are among a class of models referred to as ensembles since they are a collection of underlying models or learners. In the case of Random Forests, all of the underlying models are Decision Trees so this would be a homogeneous ensemble, while an ensemble of differing underlying models would unsurprisingly be referred to as heterogeneous. Regardless of whether the ensemble is homo- or heterogeneous though, the ensemble performs best when the underlying models are not correlated (i.e. make mistakes in different places).


In [ ]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None,
                            oob_score=True)
clf.fit(x_train, y_train)
train_acc = clf.score(x_train,y_train)
test_acc = clf.score(x_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Out-of-bag Accuracy: {clf.oob_score_}')
print(f'Test Accuracy: {test_acc}')
plot_decision_boundary(x_test,y_test,clf)

### n << p

A by product of the algorithmic choice to subsample predictors for each split, is that Random Forests can be trained on datasets with a large number of variables, even if the number of variables far exceeds the number of predictors.

Note: This is only a computational benefit and does not necessarily help with the overfitting assocaited with $n<<p$ scenarios.

In [ ]:
# Source: https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq
df = pd.read_csv("/Users/sharad/Courses/DATA 5610/Data/TCGA-PANCAN-HiSeq-801x20531/data.csv")
df.drop(columns=['Unnamed: 0'],inplace=True)
labels = pd.read_csv("/Users/sharad/Courses/DATA 5610/Data/TCGA-PANCAN-HiSeq-801x20531/labels.csv")
df.head()

In [ ]:
X = df.to_numpy()
y = labels.to_numpy()
y = y[:,1]

print(X.shape)
print(y.shape)
del(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None,
                            max_features=25,
                            oob_score=True)
clf.fit(X_train,y_train)

train_acc = clf.score(X_train,y_train)
test_acc = clf.score(X_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Out-of-bag Accuracy: {clf.oob_score_}')
print(f'Test Accuracy: {test_acc}')

### Sensitivity to hyperparmeters

Another benefit of Random Forests to the Machine Learning practitioner is that they are robust to the selection of their hyperparameters, making them an easy first choice for model selection.

In [ ]:
data = fetch_olivetti_faces()
X, y = data.data, data.target
mask = y < 30
X = X[mask]
y = y[mask]

print(X.shape)

In [ ]:
n_trees = [1,10,25,100,500,1000,5000]

oob_acc = []
for n in n_trees:
    clf = RandomForestClassifier(n_estimators=n,
                                 max_depth=None,
                                 max_features=25,
                                 oob_score=True)
    clf.fit(X,y)
    oob_acc.append(clf.oob_score_)

plt.grid()
plt.ylim([0, 1.0])
plt.plot(n_trees, oob_acc)
plt.xlabel("Number of Trees")
plt.ylabel("OOB Accuracy")
print(oob_acc)

In [ ]:
n_features_per_split = [1,10,25,50,100,500]

oob_acc = []
for n in n_features_per_split:
    clf = RandomForestClassifier(n_estimators=100,
                                 max_depth=None,
                                 max_features=n,
                                 oob_score=True)
    clf.fit(X,y)
    oob_acc.append(clf.oob_score_)

plt.grid()
plt.ylim([0, 1.0])
plt.plot(n_features_per_split, oob_acc)
plt.xlabel("Number of Features")
plt.ylabel("OOB Accuracy")
print(oob_acc)

In [ ]:
max_depth = [1,5,10,100,1000,5000]

oob_acc = []
for n in max_depth:
    clf = RandomForestClassifier(n_estimators=100,
                                 max_depth=n,
                                 max_features=25,
                                 oob_score=True)
    clf.fit(X,y)
    oob_acc.append(clf.oob_score_)

plt.grid()
plt.ylim([0, 1.0])
plt.plot(max_depth, oob_acc)
plt.xlabel("Max Depth")
plt.ylabel("OOB Accuracy")
print(oob_acc)

# AdaBoost

An alternate approach to ensembling trees that grows the ensemble sequentially is called Boosting. Boosting has become the go-to method for improving classification accuracy on structured datasets, as evidenced by many of the top Kaggle models. This increase in accuracy does come at the cost of explainability but if accuracy is what matters, these are a strong candidate.

Arguably the simplest boosting algorithm to understand is AdaBoost (short for Adaptive Boosting). The underlying motivations of this algorithm carry over in to many of the more advanced methods, so a good understanding of AdaBoost is critical.

Rather than fitting trees to max depth, as with Random Forests, AdaBoost takes a different extreme by fitting what is referred to as a Decision Stump, a tree consisting of a single split. 
To grow this stump, 
1. Identical weights are given to all of the observations and a weighted sample is performed to select the data for that split. 
2. The split can then be found identically to how CART would, by sorting and searching through all predictors. The Decision Stump is often referred to as a weak learner, since it will likely have a very high error rate due to severely underfitting the data. 
3. For the next tree though, the training data will be passed through the stump and misclassified observations will be upweighted, while correctly classified examples are downweighted.
4. Resample with the adjusted (adapted) weights and grow another stump.
5. Repeat until some stopping criterion is reached (e.g. no errors on training data).

### Hyperparameters
- Max number of stumps: The maximum number of stumps to train before ending training.
- Learning rate: Weight applied to each classifier at each boosting iteration. A higher learning rate increases the contribution of each classifier.

### Disadvantages
- Must be grown sequentially (i.e. not easily parallelizable)
- Sensitive to outliers

In [ ]:
df = pd.read_csv('/Users/sharad/Courses/DATA 5610/Data/pima-indians-diabetes.csv', names=['Pregnancies',
                                                                                           'Glucose',
                                                                                           'BP',
                                                                                           'SkinThickness',
                                                                                           'Insulin',
                                                                                           'BMI',
                                                                                           'DiabetesPedigreeFunction',
                                                                                           'Age',
                                                                                           'Class'])
df

In [ ]:
X = df.iloc[:,0:8]
y = df.iloc[:,8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
X_train.shape

In [ ]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None,
                            oob_score=True)
clf.fit(X_train,y_train)

train_acc = clf.score(X_train,y_train)
test_acc = clf.score(X_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Out-of-bag Accuracy: {clf.oob_score_}')
print(f'Test Accuracy: {test_acc}')

In [ ]:
clf = AdaBoostClassifier(n_estimators=1000, learning_rate=0.01)

clf.fit(X_train,y_train)
train_acc = clf.score(X_train,y_train)
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
test_acc = clf.score(X_test, y_test)

print(f'Training Accuracy: {train_acc}')
print(f'Cross-validated Accuracy: {np.mean(cv_scores)}')
print(f'Test Accuracy: {test_acc}')

# XGBoost

Extreme Gradient Boosting (XGBoost) has become one of the most popular algorithms for predictive modeling on structured datasets. Gradient Boosting Machines (GBM) underly XGBoost and can be viewed themselves as a generalization of AdaBoost, where the boosting procedure is structured as an optimization problem with an objective of minimizing the loss by adding weak learners, resembling something like gradient descent.

GBMs require the user to define a differentiable loss function such as MSE or cross-entropy based on their target or outcome variable. Then weak learners are added to the model such that they decrease this loss (i.e. follow the gradient) until some stopping criterion is reached.

XGBoost extends these properties a bit more by adding shrinkage and feature subsampling. Shrinkage scales newly added weights by a factor $\eta$ after each iteration. Similar to a learning rate in stochastic optimization, shrinkage reduces the influence of each tree and leaves space for future trees to improve the model. Feature subsampling is identical to the subsampling of predictors as performed in Random Forests and serves the same purpose of decorellating the trees.

Most of the other improvements associated with XGBoost are computational as a main focus of the algorithm was improved computational efficiency. These improvements include: efficient splitting algorithms, parallelization of tree construction, cache-aware access, distributed computing.

Source: https://www.mygreatlearning.com/blog/xgboost-algorithm/

In [ ]:
clf = XGBClassifier(use_label_encoder=False, 
                    eval_metric='error', 
                    n_estimators= 2000,
                    max_depth= 9,
                    min_child_weight= 2,
                    gamma=0.4,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective= 'binary:logistic',
                    nthread= -1,
                    scale_pos_weight=1)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train,y_train)
test_acc = clf.score(X_test, y_test)

In [ ]:
print(f'Training Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

## Cross-validation

In [ ]:
X, y = make_circles(n_samples=100, noise=0.2, factor=0.01, random_state=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
folds = np.random.randint(0,10,X_train.shape[0])
folds

In [ ]:
val_fold_preds = []
val_fold_true = []

for i in range(10):
    train_fold_x = X_train[folds != i,:]
    train_fold_y = y_train[folds != i]
    val_fold_x = X_train[folds == i,:]
    val_fold_y = y_train[folds == i]
    
    clf = DecisionTreeClassifier().fit(train_fold_x, train_fold_y)
    val_fold_preds.extend(clf.predict(val_fold_x))
    val_fold_true.extend(val_fold_y)

confusion_matrix(val_fold_true, val_fold_preds)

## Feature Importance

Given the out-of-the-box usability and natural handling of collinearity, Random Forests are a strong candidate for performing feature selection when presented with a large number of variables or simply for a post hoc feature importance assessment. There are two common ways to assess feature importance for Random Forests: impurity based and permutation based.

### Impurity Based

When training a Decision Tree, the splits are chosen to decrease the impurity in child nodes. Feature importance is calculated as the decrease in node impurity weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The higher the value the more important the feature. For Random Forests, this is average across all trees resulting in an estimate of importance for each feature in the dataset.

In [ ]:
colnames = ['Pregnancies','Glucose','BP','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Class']
df = pd.read_csv('/Users/sharad/Courses/DATA 5610/Data/pima-indians-diabetes.csv', names=colnames)

X = df.iloc[:,0:8]
y = df.iloc[:,8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None,
                            oob_score=True)
clf.fit(X_train,y_train)

In [ ]:
plt.bar(x=colnames[:8],height=clf.feature_importances_)
plt.xticks(rotation = 90)
plt.show()

### Permutation Based

The impurity based approach has a few drawbacks that are worth considering though. First, it can bias towards high cardinality or continuous features due to them having a large number of possible splits and sometimes being over represented in a tree. This approach also only estimates importance based on training data so an overfit model may provide incorrect feature importance.

The permutation based approach accounts for these shortcomings and provides a few more benefits as well. One major strength is that this approach is more general in that it can be applied to any supervised learning model. To estimate the importance of a feature, one-by-one, the values of each feature are randomly shuffled and passed back through the trained model. The decrease in the model performance when that feature is shuffled is deemed the "importance". This shuffling is typically repeated multiple times and averaged for each variable. 

A strength of this approach is that it can be estimated on holdout or test data, which allows overfitting to the training set to be accounted for. It is common to estimate the permutation based feature importance via cross-validation if compute resources allow.

In [ ]:
cv_perm_imp = []
folds = np.random.randint(0,10,X_train.shape[0])

for i in range(10):
    train_fold_x = X_train[folds != i]
    train_fold_y = y_train[folds != i]
    val_fold_x = X_train[folds == i]
    val_fold_y = y_train[folds == i]
    
    clf = RandomForestClassifier().fit(train_fold_x, train_fold_y)
    
    perm_imp = permutation_importance(clf, val_fold_x, val_fold_y, n_repeats = 30, random_state=24)
    cv_perm_imp.append(perm_imp['importances_mean'])

cv_perm_imp = np.stack(cv_perm_imp)

In [ ]:
plt.figure(figsize=(10, 10))
plt.boxplot(cv_perm_imp, labels = colnames[:8], showmeans=True)
plt.xticks(rotation = 90)
plt.show()

## Partial Dependence Plots

Partial Dependence Plots (PDP) give us another way of interpreting the model's behavior. The goal of PDPs though is not importance though but rather the change of the target variable for changing values of one or more input variables. The main output of this approach is a visualization of this relationship, therefore, these plots are constrained to one- or two-way plots, which means we can either look at the effect of one variable on the target variable or on the interaction between two variables. Due to this constraint, it can be valuable to assess the most important features first and then visualize these via PDPs.

At a high level, PDPs are created by holding all features in the model fixed except for the feature of interest. The feature of interest is then varied along the range of observed values and the predicted output is stored. This can be repeated for varying values of the fixed input features and all results averaged to give a final plot of the typical change in the target for changes in the input feature. A similar process is taken for the two-way PDP.

The biggest issue of PDPs are that they assume features are truly independent. This is a very difficult assumption to confirm in real data so all PDPs should be interpreted with this caveat in mind.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

features = [1, 5]
PartialDependenceDisplay.from_estimator(clf, X_train, features)

In [ ]:
features = [(1, 5)]
PartialDependenceDisplay.from_estimator(clf, X_train, features)